In [1]:
# Import Dependencies
from pymongo import MongoClient
from config import cloudM, cloudMpassword
import pandas as pd
import numpy as np
import datetime as dt
import csv
import re

In [2]:
#create a variable for columns to keep
#did not include the note columns because it would drop some data due to null
columns = ["Country", "Geography: Geographic coordinates", "Geography: Map references", "Geography: Area - total",
            "Economy: Real GDP (purchasing power parity)", 
            "Economy: Real GDP growth rate",
            "Economy: Real GDP per capita",
            "Economy: Current account balance",
            "Economy: Exports",
            "Economy: Exports - partners",
            "Economy: Reserves of foreign exchange and gold",
            "Energy: Electricity access - electrification - total population",
            "Economy: Credit ratings - Moody's rating",
            "Economy: Credit ratings - Standard & Poors rating",
            "Economy: Credit ratings - Fitch rating",
            "Economy: Unemployment rate",
            "Economy: Population below poverty line",
            "Economy: Household income or consumption by percentage share - highest 10%",
            "People and Society: Population",
            "People and Society: Nationality - noun",
            "People and Society: Nationality - adjective",
            "People and Society: Languages",
            "People and Society: Religions",
            "People and Society: Age structure - 0-14 years",
            "People and Society: Age structure - 15-24 years",
            "People and Society: Age structure - 25-54 years",
            "People and Society: Age structure - 55-64 years",
            "People and Society: Median age - total",
            "People and Society: Median age - male",
            "People and Society: Median age - female",
            "People and Society: Population growth rate",
            "People and Society: Birth rate",
            "People and Society: Death rate",
            "People and Society: Net migration rate",
            "People and Society: Urbanization - urban population",
            "People and Society: Sex ratio - at birth",
            "People and Society: Sex ratio - 65 years and over",
            "People and Society: Sex ratio - total population",
            "People and Society: Infant mortality rate - total",
            "People and Society: Life expectancy at birth - total population",
            "People and Society: Life expectancy at birth - male",
            "People and Society: Life expectancy at birth - female",
            "People and Society: Total fertility rate",
            "People and Society: Obesity - adult prevalence rate",
            "People and Society: Tobacco use - total",
            "Communications: Internet users - total",
            "Communications: Internet users - percent of population",
            "Military and Security: Military expenditures",
            "Military and Security: Military service age and obligation",
            "Terrorism: Terrorist group(s)", "Military and Security: Military - note"]        

In [3]:
#load data and create df to clean
data = pd.read_csv("countries.csv")
#previouly determined all fields were objects, load as string for cleaning
df = pd.DataFrame(data).astype("string")
df = df.loc[:, columns].copy()
#check result
df.head(2)

,Country,Geography: Geographic coordinates,Geography: Map references,Geography: Area - total,Economy: Real GDP (purchasing power parity),Economy: Real GDP growth rate,Economy: Real GDP per capita,Economy: Current account balance,Economy: Exports,Economy: Exports - partners,...,People and Society: Life expectancy at birth - female,People and Society: Total fertility rate,People and Society: Obesity - adult prevalence rate,People and Society: Tobacco use - total,Communications: Internet users - total,Communications: Internet users - percent of population,Military and Security: Military expenditures,Military and Security: Military service age and obligation,Terrorism: Terrorist group(s),Military and Security: Military - note
0,Afghanistan,"33 00 N, 65 00 E",Asia,"652,230 sq km",$77.04 billion (2020 est.)$78.56 billion (2019...,2.7% (2017 est.)2.2% (2016 est.)1% (2015 est.),"$2,000 (2020 est.)$2,100 (2019 est.)$2,000 (20...",$1.014 billion (2017 est.)$1.409 billion (2016...,$1.48 billion (2020 est.),"United Arab Emirates 45%, Pakistan 24%, India ...",...,55.28 years (2022 est.),4.62 children born/woman (2022 est.),5.5% (2016),23.3% (2020 est.),"7,007,101 (2020 est.)",18% (2020 est.),3.3% of GDP (2019) (approximately $2.35 billio...,not available,Haqqani Taliban Network; Harakat ul-Mujahidin;...,"as of early 2022, the Taliban’s primary securi..."
1,Akrotiri,"34 37 N, 32 58 E",Middle East,123 sq km,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,defense is the responsibility of the UK; Akrot...


In [4]:
#drop null values & determing number of rows 
df = df.dropna()
len(df)

48

In [5]:
#set index to country & check result
df.set_index('Country')
df.head(2)

,Country,Geography: Geographic coordinates,Geography: Map references,Geography: Area - total,Economy: Real GDP (purchasing power parity),Economy: Real GDP growth rate,Economy: Real GDP per capita,Economy: Current account balance,Economy: Exports,Economy: Exports - partners,...,People and Society: Life expectancy at birth - female,People and Society: Total fertility rate,People and Society: Obesity - adult prevalence rate,People and Society: Tobacco use - total,Communications: Internet users - total,Communications: Internet users - percent of population,Military and Security: Military expenditures,Military and Security: Military service age and obligation,Terrorism: Terrorist group(s),Military and Security: Military - note
10,Argentina,"34 00 S, 64 00 W",South America,"2,780,400 sq km",$893.31 billion (2020 est.)$991.52 billion (20...,-2.03% (2019 est.)-2.53% (2018 est.)2.83% (201...,"$19,700 (2020 est.)$22,100 (2019 est.)$22,800 ...",-$3.997 billion (2019 est.)-$27.049 billion (2...,$64.18 billion (2020 est.) note: data are in c...,"Brazil 16%, China 11%, United States 7%, Chile...",...,81.59 years (2022 est.),2.18 children born/woman (2022 est.),28.3% (2016),24.5% (2020 est.),"39,024,016 (2020 est.)",86% (2020 est.),0.8% of GDP (2021 est.)0.8% of GDP (2020)0.7% ...,18-24 years of age for voluntary military serv...,Hizballah,Argentina has Major Non-NATO Ally (MNNA) statu...
15,Austria,"47 20 N, 13 20 E",Europe,"83,871 sq km",$463.12 billion (2020 est.)$495.8 billion (201...,1.42% (2019 est.)2.58% (2018 est.)2.4% (2017 e...,"$51,900 (2020 est.)$55,800 (2019 est.)$55,300 ...",$12.667 billion (2019 est.)$5.989 billion (201...,$226.79 billion (2020 est.) note: data are in ...,"Germany 28%, United States 7%, Italy 6%, Switz...",...,85.04 years (2022 est.),1.51 children born/woman (2022 est.),20.1% (2016),26.4% (2020 est.),"7,846,840 (2020 est.)",88% (2020 est.),0.9% of GDP (2021 est.)0.7% of GDP (2020 est.)...,"registration requirement at age 17, the legal ...",Islamic State of Iraq and ash-Sham (ISIS),"Austria is constitutionally non-aligned, but i..."


In [6]:
#create a new DF, preserving the original in case of mistake, with shortened columns name
df_a = df.rename(columns = 
                {"Geography: Geographic coordinates" : "Geographic coordinates", "Geography: Map references" : "Map references",      
                "Economy: Real GDP (purchasing power parity)" : "Real GDP", "Economy: Real GDP growth rate" : "GDP Growth Rate", 
                "Economy: Real GDP per capita" : "GDP per Capita",
                "Economy: Current account balance": "Econ Current Acct Bal",
                "Economy: Exports" : "Exports",
                "Economy: Exports - partners" : "Exports - Partners",
                "Economy: Reserves of foreign exchange and gold" : "Foreign Exchange & Gold",
                "Energy: Electricity access - electrification - total population" :"Tota; Population Electrication",
                "Economy: Credit ratings - Moody's rating" : "Moody's Rating",
                "Economy: Credit ratings - Standard & Poors rating" : "Standard & Poors Rating",
                "Economy: Credit ratings - Fitch rating" : "Fitch Rating",
                "Economy: Unemployment rate" : "Unemployment Rate",
                "Economy: Population below poverty line" : "Population below poverty line",
                "Economy: Household income or consumption by percentage share - highest 10%" : "Household income or consumption by % -highest 10%",
                "People and Society: Population" : "Population",
                "People and Society: Nationality - noun" : "Nationality - noun",
                "People and Society: Nationality - adjective" : "Nationality - adjective",
                 "People and Society: Languages" : "Languages",
                "People and Society: Religions" : "Religions",
                "People and Society: Age structure - 0-14 years" : "Age structure - 0-14 years",
                "People and Society: Age structure - 15-24 years" : "Age structure - 15-24 years",
                "People and Society: Age structure - 25-54 years" : "Age structure - 25-54 years",
                "People and Society: Age structure - 55-64 years" : "Age structure - 55-64 years",
                "People and Society: Median age - total" : "Total Median age",
                "People and Society: Median age - male" : "Male Median age",
                "People and Society: Median age - female" : "Female Median age",
                "People and Society: Population growth rate" : "Population growth rate",
                "People and Society: Birth rate" : "Birth rate",
                "People and Society: Death rate" : "Death rate",
                "People and Society: Net migration rate" : "Net migration rate",
                "People and Society: Urbanization - urban population" : "Urban Population",
                "People and Society: Sex ratio - at birth" : "Sex ratio at Birth",
                "People and Society: Sex ratio - 65 years and over" : "65 & Older Sex Ratio",
                "People and Society: Sex ratio - total population" : "Total Pop Sex Ratio",
                "People and Society: Infant mortality rate - total" : "Total Infant Mortality Rate",
                "People and Society: Life expectancy at birth - total population" : "Total Pop Life expectancy at birth",
                "People and Society: Life expectancy at birth - male" : "Male Life expectancy at birth",
                "People and Society: Life expectancy at birth - female" : "Female Life expectancy at birth",
                "People and Society: Total fertility rate" : "Total fertility rate",
                "People and Society: Obesity - adult prevalence rate" : "Adult Obesity Prev Rate",
                "People and Society: Tobacco use - total" : "Total Tobacco use",
                "Communications: Internet users - total" : "Total Internet Users",
                "Communications: Internet users - percent of population" : "Internet users - % of Pop",
                "Military and Security: Military expenditures" : "Military expenditures",
                "Military and Security: Military service age and obligation" : "Military service age and obligation"})



In [7]:
#check results
#df_a.columns.tolist()
df_a.head(2)

,Country,Geographic coordinates,Map references,Geography: Area - total,Real GDP,GDP Growth Rate,GDP per Capita,Econ Current Acct Bal,Exports,Exports - Partners,...,Female Life expectancy at birth,Total fertility rate,Adult Obesity Prev Rate,Total Tobacco use,Total Internet Users,Internet users - % of Pop,Military expenditures,Military service age and obligation,Terrorism: Terrorist group(s),Military and Security: Military - note
10,Argentina,"34 00 S, 64 00 W",South America,"2,780,400 sq km",$893.31 billion (2020 est.)$991.52 billion (20...,-2.03% (2019 est.)-2.53% (2018 est.)2.83% (201...,"$19,700 (2020 est.)$22,100 (2019 est.)$22,800 ...",-$3.997 billion (2019 est.)-$27.049 billion (2...,$64.18 billion (2020 est.) note: data are in c...,"Brazil 16%, China 11%, United States 7%, Chile...",...,81.59 years (2022 est.),2.18 children born/woman (2022 est.),28.3% (2016),24.5% (2020 est.),"39,024,016 (2020 est.)",86% (2020 est.),0.8% of GDP (2021 est.)0.8% of GDP (2020)0.7% ...,18-24 years of age for voluntary military serv...,Hizballah,Argentina has Major Non-NATO Ally (MNNA) statu...
15,Austria,"47 20 N, 13 20 E",Europe,"83,871 sq km",$463.12 billion (2020 est.)$495.8 billion (201...,1.42% (2019 est.)2.58% (2018 est.)2.4% (2017 e...,"$51,900 (2020 est.)$55,800 (2019 est.)$55,300 ...",$12.667 billion (2019 est.)$5.989 billion (201...,$226.79 billion (2020 est.) note: data are in ...,"Germany 28%, United States 7%, Italy 6%, Switz...",...,85.04 years (2022 est.),1.51 children born/woman (2022 est.),20.1% (2016),26.4% (2020 est.),"7,846,840 (2020 est.)",88% (2020 est.),0.9% of GDP (2021 est.)0.7% of GDP (2020 est.)...,"registration requirement at age 17, the legal ...",Islamic State of Iraq and ash-Sham (ISIS),"Austria is constitutionally non-aligned, but i..."


In [8]:
#change geo coordinate to lat and long columns
df_a[["LATITUDE", "LONGITUDE"]] = df_a["Geographic coordinates"].str.split(",", expand = True)

#check result
#df_a["LATITUDE"]
#df_a["LONGITUDE"]
#df_a.columns.tolist()

## Clean GDP Columns [Real GDP]

In [9]:
# slice to make 2019 real GDP column
df_a["2020 Real GDP"] = df_a["Real GDP"].str[:15]

#remove $ from new column
df_a["2020 Real GDP"] = df_a["2020 Real GDP"].str.replace("$", "")

#remove billion
df_a["2020 Real GDP"] = df_a["2020 Real GDP"].str.replace("billion", "")

#remove comma
df_a["2020 Real GDP"] = df_a["2020 Real GDP"].str.replace(",", "")

#check result
#print(df_a["2020 Real GDP"])

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """


In [10]:
# slice to make 2019 real GDP column
df_a["2019 Real GDP"] = df_a["Real GDP"].str[28:48]

#remove $ from new column
df_a["2019 Real GDP"] = df_a["2019 Real GDP"].str.replace("$", "")

#remove billion
df_a["2019 Real GDP"] = df_a["2019 Real GDP"].str.replace("billion", "")

# #remove comma
df_a["2019 Real GDP"] = df_a["2019 Real GDP"].str.replace(",", "")

# remove (2019)
df_a["2019 Real GDP"] = df_a["2019 Real GDP"].str.replace("\(2019", "")

#remove remove letters
df_a["2019 Real GDP"] = df_a["2019 Real GDP"].str.replace("[t, e]", "") 

# remove )
df_a["2019 Real GDP"] = df_a["2019 Real GDP"].str.replace(".\)", "")

#check result
#print(df_a["2019 Real GDP"])

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """
/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  
/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: The default value of regex will change from True to False in a future version.


In [11]:
# slice to make 2018 real GDP column
df_a["2018 Real GDP"] = df_a["Real GDP"].str[52:]

#remove $ from new column
df_a["2018 Real GDP"] = df_a["2018 Real GDP"].str.replace("$", "")

# #remove billion
df_a["2018 Real GDP"] = df_a["2018 Real GDP"].str.replace("billion", "")

# # #remove comma
df_a["2018 Real GDP"] = df_a["2018 Real GDP"].str.replace(",", "")

#remove comma
df_a["2018 Real GDP"] = df_a["2018 Real GDP"].str.replace(",", "")

#remove 18.)
df_a["2018 Real GDP"] = df_a["2018 Real GDP"].str.replace("2018 est.\)", "")

#remove right facing ( paren
df_a["2018 Real GDP"] = df_a["2018 Real GDP"].str.replace("\(", "")

#remove .) 
df_a["2018 Real GDP"] = df_a["2018 Real GDP"].str.replace("[\., \)]", "")

#check result
#print(df_a["2018 Real GDP"])

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """
/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: The default value of regex will change from True to False in a future version.


In [12]:
# create new DF and drop original column
df_b = df_a.drop(["Real GDP"], axis = 1)

#check result
#df_b.columns.tolist()

## Clean GDP Columns [GDP Growth Rate]

In [13]:
# split sting for  2019 GDP Growth Rate
df_b["GDP Growth Rate Est 1 (in %)"] = df_b["GDP Growth Rate"].str[:15]

#remove the %
df_b["GDP Growth Rate Est 1 (in %)"] = df_b["GDP Growth Rate Est 1 (in %)"].str.replace("%", "")

#replace ( with space to split columns in 2nd cleaning
df_b["GDP Growth Rate Est 1 (in %)"] = df_b["GDP Growth Rate Est 1 (in %)"].str.replace("\(", " ")

# remove letters (replace with space to remove stray items in second cleaning)
df_b["GDP Growth Rate Est 1 (in %)"] = df_b["GDP Growth Rate Est 1 (in %)"].str.replace("[e, s, t)]", " ")

#check result
#df_b["GDP Growth Rate Est 1 (in %)"]

## note: clean leading and trailing items in second cleaning

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  
/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()


In [14]:
df_b["GDP Growth Rate Est 2(in %)"] = df_b["GDP Growth Rate"].str[15:31]
     
#remove the %
df_b["GDP Growth Rate Est 2(in %)"] = df_b["GDP Growth Rate Est 2(in %)"].str.replace("%", "")     

# #remove )
df_b["GDP Growth Rate Est 2(in %)"] = df_b["GDP Growth Rate Est 2(in %)"].str.replace("\)", "")

# remove letters (replace with space to remove stray items in second cleaning)
df_b["GDP Growth Rate Est 2(in %)"] = df_b["GDP Growth Rate Est 2(in %)"].str.replace("[e, s, t)]", " ")

#replace ( with space to split columns in 2nd cleaning
df_b["GDP Growth Rate Est 2(in %)"] = df_b["GDP Growth Rate Est 2(in %)"].str.replace("\(", " ")

#check result
#df_b["GDP Growth Rate Est 2(in %)"] 

## note: clean leading and trailing items in second cleaning

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys
/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.
/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]


In [15]:
df_b["GDP Growth Rate Est 3(in %)"] = df_b["GDP Growth Rate"].str[32:]

#remove the %
df_b["GDP Growth Rate Est 3(in %)"] = df_b["GDP Growth Rate Est 3(in %)"].str.replace("%", "")

# #remove )
df_b["GDP Growth Rate Est 3(in %)"] = df_b["GDP Growth Rate Est 3(in %)"].str.replace("\)", "")

#remove t
df_b["GDP Growth Rate Est 3(in %)"] = df_b["GDP Growth Rate Est 3(in %)"].str.replace("t", "")

# remove letters (replace with space to remove stray items in second cleaning)
df_b["GDP Growth Rate Est 3(in %)"] = df_b["GDP Growth Rate Est 3(in %)"].str.replace("[e, s)]", "")

#remove ( 
df_b["GDP Growth Rate Est 3(in %)"] = df_b["GDP Growth Rate Est 3(in %)"].str.replace("\(", " ")

#check result
#df_b["GDP Growth Rate Est 3(in %)"]

## note: clean leading and trailing items in second cleaning

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys
/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  app.launch_new_instance()


In [16]:
#check columns
#df_b.columns.tolist()

In [17]:
#drop GDP Growth Rate'
df_c = df_b.drop(["GDP Growth Rate"], axis = 1)

## Clean GDP Columns [GDP Per Capita]

In [18]:
#GDP per capita - 2020 column
#remove $ from original column
df_c["GDP per Capita"] = df_c["GDP per Capita"].str.replace("[$]", " ")


# slice for GDP per Capita est 2020
df_c["GDP per Capita Est 2020 (in $)"] = df_c["GDP per Capita"].str[:8]

#remove the  (
df_c["GDP per Capita Est 2020 (in $)"] = df_c["GDP per Capita Est 2020 (in $)"].str.replace("[,\(]", "")

#check results
#df_c["GDP per Capita Est 2020 (in $)"]

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.


In [19]:
# slice for GDP per Capita est 2019
df_c["GDP per Capita Est 2019 (in $)"] = df_c["GDP per Capita"].str[19:26]

#remove the  ( and comma
df_c["GDP per Capita Est 2019 (in $)"] = df_c["GDP per Capita Est 2019 (in $)"].str.replace("[,\(]", "")

#check results
#df_c["GDP per Capita Est 2019 (in $)"]

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [20]:
# slice for GDP per Capita est 2018
df_c["GDP per Capita Est 2018"] = df_c["GDP per Capita"].str[37:45]


# #remove the .)
df_c["GDP per Capita Est 2018"] = df_c["GDP per Capita Est 2018"].str.replace("\)", "")

#check result
#df_c["GDP per Capita Est 2018"] 

## remove trailing (2 in second cleaning

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [21]:
#check columns
#df_c.columns.tolist()

## Clean Economy Columns (Current Acct Bal)

In [22]:
#create slice for Econ Current Acct Bal
df_c["Econ: Current Acct Bal Est 1"] = df_c["Econ Current Acct Bal"].str[:23]

# #remove ) and replace with _ to split later
df_c["Econ: Current Acct Bal Est 1"]= df_c["Econ: Current Acct Bal Est 1"].str.replace("\(", "_")

# remove letters (replace with space to remove stray items in second cleaning)
df_c["Econ: Current Acct Bal Est 1"]= df_c["Econ: Current Acct Bal Est 1"].str.replace("[e, s, t, b, m, i, l, o, n)]", "")

#check result
#df_c["Econ: Current Acct Bal Est 1"]

## split value and year in 2nd cleaning

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [23]:
#create slice for Econ Current Acct Bal Est 2
df_c["Econ: Current Acct Bal Est 2"] = df_c["Econ Current Acct Bal"].str[27:]

# #remove ) and replace with _ to split later
df_c["Econ: Current Acct Bal Est 2"]= df_c["Econ: Current Acct Bal Est 2"].str.replace("$", "")

# #remove ) and replace with _ to split later
df_c["Econ: Current Acct Bal Est 2"]= df_c["Econ: Current Acct Bal Est 2"].str.replace("[\(]", "__")

# remove letters (replace with space to remove stray items in second cleaning)
df_c["Econ: Current Acct Bal Est 2"]= df_c["Econ: Current Acct Bal Est 2"].str.replace("[e, s, t, b, m, i, l, o, n)]", "")

#check result
#df_c["Econ: Current Acct Bal Est 2"]

## 2nd cleaning: split value and year into separate columns, remove the leading amount in #114, remove trailing period after the year

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """
/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  
/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()


In [24]:
#check progress
#df_c.head()
#df_c.columns.tolist()

## Cleaning Export Info

In [25]:
#remove $ from original column
df_c["Exports"] = df_c["Exports"].str.replace("$", "")

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


In [26]:
#create slice for export est 1
df_c["Exports Est 1"] = df_c["Exports"].str[:26]

#remove lettters
df_c["Exports Est 1"] =df_c["Exports Est 1"].str.replace("[e, s, t, b, m, i, l, o, n)]", "")

# #remove ) 
df_c["Exports Est 1"] =df_c["Exports Est 1"].str.replace("[\(]", " ")

#check results
#df_c["Exports Est 1"]
#second cleaning: split columns by value and year, remove trailing periods, format #'s to be consistent

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [27]:
#create slice for export est 2
df_c["Exports Est 2"] = df_c["Exports"].str[63:91]

#remove lettters
df_c["Exports Est 2"]=df_c["Exports Est 2"].str.replace("[e, s, t, b, m, i, l, o, n, r, a)]", "")

# #remove ) 
df_c["Exports Est 2"]=df_c["Exports Est 2"].str.replace("[\(]", " ")

#check results
#df_c["Exports Est 2"]

#second cleaning: split columns by value and year, remove trailing periods, format #'s to be consistent

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [28]:
#make slice for top export partner column
df_c["Top Export Partner- Country and %"] = df_c["Exports - Partners"].str[:15]

#check result
#df_c["Top Export Partner- Country and %"]

# Second cleaning split columns to show country and %

In [30]:
#check results
#df_c.columns.tolist()

In [44]:
#foreign exchange and gold column

# remove $ from original column
df_c['Foreign Exchange & Gold'] = df_c['Foreign Exchange & Gold'].str.replace("$", " ")

#create slice
df_c["Foreign Exchange & Gold - Est 2017, in billions"] = df_c['Foreign Exchange & Gold'].str[:9]

#remove lettters
df_c["Foreign Exchange & Gold - Est 2017, in billions"] = df_c["Foreign Exchange & Gold - Est 2017, in billions"].str.replace("[b, i, m, l]", "")

#check results
#df_c["Foreign Exchange & Gold - Est 2017, in billions"]

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.
/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.


## Clean Population Columns

In [48]:
#total population: electrification
#### create slice for electic access results are on 2020/2019
df_c["Total Population Electrification by % (Est 2019 - 2020)"] = df_c["Tota; Population Electrication"].str[:4]

#check results
#df_c["Total Population Electrification by % (Est 2019 - 2020)"]

#second cleaning remove %

In [53]:
#unemployment rate - recent result on record
#took the latest metric (results are between 2017 to 2020)
df_c["Unemployment Rate in % (Est 2012 - 2019)"] = df_c["Unemployment Rate"].str[:4]

#remove ( 
df_c["Unemployment Rate in % (Est 2012 - 2019)"] = df_c["Unemployment Rate in % (Est 2012 - 2019)"].str.replace("[\(]", " ")

#check result
#df_c["Unemployment Rate in % (Est 2012 - 2019)"]

#second cleaning remove %

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [57]:
#population below poverty line - recent result on record (results are between 2013 to 2019)
df_c["Population Below Poverty Line in % (Est 2012 - 2019)"] = df_c["Population below poverty line"].str[:4]

#check results
#df_c["Population Below Poverty Line in % (Est 2012 - 2019)"]

## remove % in second cleaning

In [63]:
# create slice for population below poverty line.  Only one set of results per country 
df_c["Percent Population Below Poverty Line - Year Recorded"] = df_c["Population below poverty line"].str[:11]

# remove ( and e
df_c["Percent Population Below Poverty Line - Year Recorded"] =df_c["Percent Population Below Poverty Line - Year Recorded"].str.replace("[\(, e]", " ")

# check results
#df_c["Percent Population Below Poverty Line - Year Recorded"]

## second cleaning: split column for value and years

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [67]:
#create slice for household income/consumption: value
df_c["Household Income or Consumption by % (top 10% of population)"] = df_c["Household income or consumption by % -highest 10%"].str[:4]

#check results
#df_c["Household Income or Consumption by % (top 10% of population)"] 

##second cleaning remove %

In [73]:
#create slice for household income/consumption: year
df_c["Household Income or Consumption by (year recorded)"] = df_c["Household income or consumption by % -highest 10%"].str[5:]

# remove (), .  and letters
df_c["Household Income or Consumption by (year recorded)"]  = df_c["Household Income or Consumption by (year recorded)"].str.replace("[\(\), \.\, e, s, t]", "")

#check results
#df_c["Household Income or Consumption by (year recorded)"] 

# Second cleaning change FY2013 to 2013

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [79]:
# create population as of 2022
df_c["Population (as of 2022)"] = df_c["Population"].str[:13]

#check results
#df_c["Population (as of 2022)"] 

# second cleaning remove trailing ( and trailing #'s

In [81]:
#check columns
#df_c.columns.tolist()

In [84]:
#slice for percent 14 and under
df_c["Percent of Population 14 yrs and under"] = df_c["Age structure - 0-14 years"].str[:5]

#check results
#df_c["Percent of Population 14 yrs and under"]

In [89]:
#percent of males 14 and under

df_c["Percent of Males 14 yrs and Under"] = df_c["Age structure - 0-14 years"].str[12:24]

# remove / and letters
df_c["Percent of Males 14 yrs and Under"]   = df_c["Percent of Males 14 yrs and Under"] .str.replace("[/, f, e, m]", "")

#check results
#df_c["Percent of Males 14 yrs and Under"]

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [93]:
#slice for females 14 and under
df_c["Percent of Females 14 yrs and Under"] = df_c["Age structure - 0-14 years"].str[28:]

# remove / and letters
df_c["Percent of Females 14 yrs and Under"] = df_c["Percent of Females 14 yrs and Under"]  .str.replace("[\), f, e, m, a, l]", "")

#check result
#df_c["Percent of Females 14 yrs and Under"]

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [95]:
#slice for percent 15- 24 and under
df_c["Percent of People 15-24 yrs and Under"] = df_c["Age structure - 15-24 years"].str[:5]

#check results
#df_c["Percent of People 15-24 yrs and Under"] 

In [102]:
#slice for males 15-24 and under
df_c["Males Between 15-24 Years"] = df_c["Age structure - 15-24 years"].str[12:22]

# remove / and letters
df_c["Males Between 15-24 Years"] = df_c["Males Between 15-24 Years"].str.replace("[/, f, e, m]", "")

#check results
#df_c["Males Between 15-24 Years"] 

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [103]:
#slice for fmales 15-24 and under
df_c["Females Between 15-24 Years"] = df_c["Age structure - 15-24 years"].str[28:]

# remove / and letters
df_c["Females Between 15-24 Years"]  = df_c["Females Between 15-24 Years"].str.replace("[\), f, e, m]", "")

#check results
#df_c["Females Between 15-24 Years"] 

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [105]:
#slice for percent 25 - 54 and under
df_c["Percent of People 25-54 yrs"] = df_c["Age structure - 25-54 years"].str[:5]

#check results
#df_c["Percent of People 25-54 yrs"] 

In [109]:
#slice for males 25-54 
df_c["Males Between 25-54 yrs"] = df_c["Age structure - 25-54 years"].str[11:23]

# remove / and letters
df_c["Males Between 25-54 yrs"]  = df_c["Males Between 25-54 yrs"].str.replace("[/, f, e, m]", "")

#check result
#df_c["Males Between 25-54 yrs"] 

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [115]:
#slice for females 25-54 and under
df_c["Females Between 25-54 yrs"] = df_c["Age structure - 25-54 years"].str[28:]

# remove / and letters
df_c["Females Between 25-54 yrs"] = df_c["Females Between 25-54 yrs"].str.replace("[\), f, e, m]", "")

#check results
#df_c["Females Between 25-54 yrs"]

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [119]:
#slice for percent 55-64
df_c["Percent of People 55-64"] = df_c["Age structure - 55-64 years"].str[:5]

# remove ( and l%
df_c["Percent of People 55-64"] = df_c["Percent of People 55-64"].str.replace("[\(, %]", "")

#check results
#df_c["Percent of People 55-64"]


/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [122]:
#slice for males 55-64
df_c["Males Between 55-64 yrs"] = df_c["Age structure - 55-64 years"].str[11:23]

# remove / and letters
df_c["Males Between 55-64 yrs"]  = df_c["Males Between 55-64 yrs"].str.replace("[/, f, e, m]", "")

#Check Results
#df_c["Males Between 55-64 yrs"]

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [125]:
#slice for females 55-64
df_c["Females Between 55-64 yrs"] = df_c["Age structure - 55-64 years"].str[28:]

# remove / and letters
df_c["Females Between 55-64 yrs"] = df_c["Females Between 55-64 yrs"].str.replace("[\), f, e, m]", "")

#check Results
#df_c["Females Between 55-64 yrs"] 

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [127]:
#slice total median age column
df_c["Median Age (in years)"] = df_c["Total Median age"].str[:4]

#check results
#df_c["Median Age (in years)"] 

In [129]:
#female median age column
df_c["Female Median Age (in years)"] = df_c["Female Median age"].str[:4]

#check results
#df_c["Female Median Age (in years)"]

In [131]:
# male median age column
df_c["Male Median Age (in years)"] = df_c["Male Median age"].str[:4]

#check results
#df_c["Male Median Age (in years)"] 

In [135]:
#pop growth rate column
df_c["Population Growth Rate (Est 2022)"] = df_c["Population growth rate"].str[:5]

#remove %
df_c["Population Growth Rate (Est 2022)"] = df_c["Population Growth Rate (Est 2022)"].str.replace("%", "")

#check results
#df_c["Population Growth Rate (Est 2022)"]

In [137]:
#birth rate column
df_c["Birth rate (births/1,000 population, 2022 est)"] = df_c["Birth rate"].str[:5]

#check results
#df_c["Birth rate (births/1,000 population, 2022 est)"]

In [139]:
#Gender rate at birth 
df_c["Gender Ratio at Birth (male(s)/female)"] = df_c["Sex ratio at Birth"].str[:5]

#check results
#df_c["Gender Ratio at Birth (male(s)/female)"]

In [142]:
#sex ratio 65 and older
df_c["65 & Older Gender Ratio (male(s)/female)"] = df_c["65 & Older Sex Ratio"].str[:5]

#remove m
df_c["65 & Older Gender Ratio (male(s)/female)"] = df_c["65 & Older Gender Ratio (male(s)/female)"].str.replace("m", "")

#check results
#df_c["65 & Older Gender Ratio (male(s)/female)"] 

In [147]:
#sex ratio total population
df_c["Total Pop Gender Ratio (male(s)/female, as of 2022"] = df_c["Total Pop Sex Ratio"].str[:4]

#remove letters
df_c["Total Pop Gender Ratio (male(s)/female, as of 2022"] = df_c["Total Pop Gender Ratio (male(s)/female, as of 2022"].str.replace("[m,a]", "")

#check results
#df_c["Total Pop Gender Ratio (male(s)/female, as of 2022"]

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [150]:
#infant mortality rate column
df_c["Total Infant Mortality Rate (deaths/1,000 live births)"] = df_c["Total Infant Mortality Rate"].str[:5]

#remove d
#check results
df_c["Total Infant Mortality Rate (deaths/1,000 live births)"] = df_c["Total Infant Mortality Rate (deaths/1,000 live births)"].str.replace("d", "")

#check results
#df_c["Total Infant Mortality Rate (deaths/1,000 live births)"] 

In [153]:
# life expectancy at birth
df_c["Total Population Life Expectancy at Birth (in years)"] = df_c["Total Pop Life expectancy at birth"].str[:5]

#remove letters
df_c["Total Population Life Expectancy at Birth (in years)"] = df_c["Total Population Life Expectancy at Birth (in years)"].str.replace("[y, e]", "")

#check results
#df_c["Total Population Life Expectancy at Birth (in years)"]

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [155]:
#male life expectancy at birth
df_c["Male Life expectancy at birth (in years)"] = df_c["Male Life expectancy at birth"].str[:5]

#check results
#df_c["Male Life expectancy at birth (in years)"] 

In [157]:
df_c["Female Life expectancy at birth (in years)"] = df_c["Female Life expectancy at birth"].str[:5]

#check results
#df_c["Female Life expectancy at birth (in years)"]

In [160]:
#Total fertility rate column
df_c["Total Fertility Rate (children born/woman (2022 est.)"] = df_c["Total fertility rate"].str[:5]

#remove letters chi
df_c["Total Fertility Rate (children born/woman (2022 est.)"] = df_c["Total Fertility Rate (children born/woman (2022 est.)"].str.replace("[c,h,i]", "")
#check results
#df_c["Total Fertility Rate (children born/woman (2022 est.)"]

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [164]:
#obesity rate column
df_c["Percent of Adult Obesity Rate (Est 2016)"] = df_c["Adult Obesity Prev Rate"].str[:5]

#remove % and (
df_c["Percent of Adult Obesity Rate (Est 2016)"] = df_c["Percent of Adult Obesity Rate (Est 2016)"].str.replace("[%, \()]", "")

#check results
#df_c["Percent of Adult Obesity Rate (Est 2016)"] 

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [167]:
#tobacco use
df_c["Percent of Tobacco Use (Est 2020)"] = df_c["Total Tobacco use"].str[:5]

#remove % and (
df_c["Percent of Tobacco Use (Est 2020)"] = df_c["Percent of Tobacco Use (Est 2020)"].str.replace("[%, \()]", "")

#check results
#df_c["Percent of Tobacco Use (Est 2020)"]

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [170]:
#internet users column
df_c["Percent of Internet Users (2020 est.)"] = df_c["Total Internet Users"].str[:10]

#remove mill
df_c["Percent of Internet Users (2020 est.)"] = df_c["Percent of Internet Users (2020 est.)"].str.replace("[m, i, l]", "")

#check results
#df_c["Percent of Internet Users (2020 est.)"] 

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [173]:
#2021 military expenditures
df_c["2021 Military Expenditures (as % of GDP)"] = df_c["Military expenditures"].str[:3]

#remove %
df_c["2021 Military Expenditures (as % of GDP)"] = df_c["2021 Military Expenditures (as % of GDP)"].str.replace("%", "")

#check results
#df_c["2021 Military Expenditures (as % of GDP)"]

In [179]:
#2020 military expenditures
df_c["2020 Military Expenditures (as % of GDP)"] = df_c["Military expenditures"].str[20:29]

#remove t, , ), %, o, f, g, d
df_c["2020 Military Expenditures (as % of GDP)"] = df_c["2020 Military Expenditures (as % of GDP)"].str.replace("[t, \), %, o, g, d, G, D, P, f]", "")

#check results
#df_c["2020 Military Expenditures (as % of GDP)"]

#second cleaning fix last result

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [188]:
#2019 military expenditures
df_c["2019 Military Expenditures (as % of GDP)"] = df_c["Military expenditures"].str[39:51]

#remove letters and items
df_c["2019 Military Expenditures (as % of GDP)"] = df_c["2019 Military Expenditures (as % of GDP)"].str.replace("[t, o, f, g, d, p, G, D, P, e, s]", "")

#check results
#df_c["2019 Military Expenditures (as % of GDP)"]

#Second cleaning: remove percent, leading and traiing items

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [190]:
#drop dup columns and organize into new DF
df_d = df_c.drop(['GDP per Capita',
     'Econ Current Acct Bal',
     'Exports',
     'Exports - Partners',
     'Foreign Exchange & Gold',
     'Tota; Population Electrication',
     "Moody's Rating",
     'Standard & Poors Rating',
     'Fitch Rating',
     'Unemployment Rate',
     'Population below poverty line',
     'Household income or consumption by % -highest 10%',
     'Population',
     'Nationality - noun',
     'Nationality - adjective',
     'Languages',
     'Religions',
     'Age structure - 0-14 years',
     'Age structure - 15-24 years',
     'Age structure - 25-54 years',
     'Age structure - 55-64 years',
     'Total Median age',
     'Male Median age',
     'Female Median age',
     'Population growth rate',
     'Birth rate',
     'Death rate',
     'Net migration rate',
     'Urban Population',
     'Sex ratio at Birth',
     '65 & Older Sex Ratio',
     'Total Pop Sex Ratio',
     'Total Infant Mortality Rate',
     'Total Pop Life expectancy at birth',
     'Male Life expectancy at birth',
     'Female Life expectancy at birth',
     'Total fertility rate',
     'Adult Obesity Prev Rate',
     'Total Tobacco use',
     'Total Internet Users',
     'Internet users - % of Pop',
     'Military expenditures'], axis=1)


In [191]:
#check results
df_d.columns.to_list()

['Country',
 'Geographic coordinates',
 'Map references',
 'Geography: Area - total',
 'Military service age and obligation',
 'Terrorism: Terrorist group(s)',
 'Military and Security: Military - note',
 'LATITUDE',
 'LONGITUDE',
 '2020 Real GDP',
 '2019 Real GDP',
 '2018 Real GDP',
 'GDP Growth Rate Est 1 (in %)',
 'GDP Growth Rate Est 2(in %)',
 'GDP Growth Rate Est 3(in %)',
 'GDP per Capita Est 2020 (in $)',
 'GDP per Capita Est 2019 (in $)',
 'GDP per Capita Est 2018',
 'Econ: Current Acct Bal Est 1',
 'Econ: Current Acct Bal Est 2',
 'Exports Est 1',
 'Exports Est 2',
 'Top Export Partner- Country and %',
 'Foreign Exchange & Gold - Est 2017',
 'Foreign Exchange & Gold - Est 2017, in billions',
 'Total Population Electrification by % (Est 2019 - 2020)',
 'Unemployment Rate (Est 2012 - 2019)',
 'Unemployment Rate in % (Est 2012 - 2019)',
 'Population Below Poverty Line in % (Est 2012 - 2019)',
 'Population below poverty line - Year Recorded',
 'Percent Population Below Poverty Lin

In [192]:
# reorder columns

df_d = df_d[['Country',
 'Geographic coordinates',
 'Map references',
 'Geography: Area - total',
 'LATITUDE', 'LONGITUDE',
 'Terrorism: Terrorist group(s)',
 'LONGITUDE',
 '2020 Real GDP',
 '2019 Real GDP',
 '2018 Real GDP',
 'GDP Growth Rate Est 1 (in %)',
 'GDP Growth Rate Est 2(in %)',
 'GDP Growth Rate Est 3(in %)',
 'GDP per Capita Est 2020 (in $)',
 'GDP per Capita Est 2019 (in $)',
 'GDP per Capita Est 2018',
 'Econ: Current Acct Bal Est 1',
 'Econ: Current Acct Bal Est 2',
 'Exports Est 1',
 'Exports Est 2',
 'Top Export Partner- Country and %',
 'Foreign Exchange & Gold - Est 2017',
 'Foreign Exchange & Gold - Est 2017, in billions',
 'Total Population Electrification by % (Est 2019 - 2020)',
 'Unemployment Rate (Est 2012 - 2019)',
 'Unemployment Rate in % (Est 2012 - 2019)',
 'Population Below Poverty Line in % (Est 2012 - 2019)',
 'Population below poverty line - Year Recorded',
 'Percent Population Below Poverty Line - Year Recorded',
 'Household income or consumption by % (-highest 10%)',
 'Household Income or Consumption by % -Est 2010 - 2017 (highest 10% of population)',
 'Household Income or Consumption by (year recorded)',
 'Population (as of 2022)',
 'Percent of People 14 yrs and under',
 'Percent of Population 14 yrs and under',
 'Percent of Males 14 yrs and Under',
 'Percent of Females 14 yrs and Under',
 'Percent of People 15-24 yrs and Under',
 'Males Between 15-24 Years',
 'Females Between 15-24 Years',
 'Percent of People 25-54 yrs',
 'Males Between 25-54 yrs',
 'Females Between 25-54 yrs',
 'Percent of People 55-64',
 'Males Between 55-64 yrs',
 'Females Between 55-64 yrs',
 'Median Age (in years)',
 'Female Median Age (in years)',
 'Male Median Age (in years)',
 'Population Growth Rate (Est 2022)',
 'Birth rate (births/1,000 population, 2022 est)',
 'Sex ratio at Birth (male(s)/female)',
 'Sex Ratio at Birth (male(s)/female)',
 '65 & Older Sex Ratio (male(s)/female)',
 'Total Pop Gender Ratio (male(s)/female, as of 2022',
 'Total Infant Mortality Rate (deaths/1,000 live births)',
 'Total Population Life Expectancy at Birth (in years)',
 'Male Life expectancy at birth (in years)',
 'Female Life expectancy at birth (in years)',
 'Total Fertility Rate (children born/woman (2022 est.)',
 'Adult Obesity Prev Rate in percentages (Est 2016)',
 'Percent of Adult Obesity Rate (Est 2016)',
 'Percent of Total Tobacco Use (Est 2020)',
 'Percent of Internet Users (2020 est.)',
 '2021 Military Expenditures (as % of GDP)',
 '2020 Military Expenditures (as % of GDP)',
 '2019 Military Expenditures (as % of GDP)', 
 'Military and Security: Military - note',
 'Military service age and obligation']]

In [194]:
#check result
#df_d.columns.to_list()
df_d.head()

,Country,Geographic coordinates,Map references,Geography: Area - total,LATITUDE,LONGITUDE,Terrorism: Terrorist group(s),LONGITUDE,2020 Real GDP,2019 Real GDP,...,Total Fertility Rate (children born/woman (2022 est.),Adult Obesity Prev Rate in percentages (Est 2016),Percent of Adult Obesity Rate (Est 2016),Percent of Total Tobacco Use (Est 2020),Percent of Internet Users (2020 est.),2021 Military Expenditures (as % of GDP),2020 Military Expenditures (as % of GDP),2019 Military Expenditures (as % of GDP),Military and Security: Military - note,Military service age and obligation
10,Argentina,"34 00 S, 64 00 W",South America,"2,780,400 sq km",34 00 S,64 00 W,Hizballah,64 00 W,893.31,991.52,...,2.18,28.3,28.3,24.5,39024016,0.8,.0.8,0)0.7%,Argentina has Major Non-NATO Ally (MNNA) statu...,18-24 years of age for voluntary military serv...
15,Austria,"47 20 N, 13 20 E",Europe,"83,871 sq km",47 20 N,13 20 E,Islamic State of Iraq and ash-Sham (ISIS),13 20 E,463.12,495.8,...,1.51,20.1,20.1,26.4,7846840,0.9,.0.7,0.)0.7%,"Austria is constitutionally non-aligned, but i...","registration requirement at age 17, the legal ..."
16,Azerbaijan,"40 30 N, 47 30 E",Asia,"86,600 sq km",40 30 N,47 30 E,Islamic State of Iraq and ash-Sham (ISIS); Isl...,47 30 E,138.51,144.74,...,1.86,19.9,19.9,24,8579153,5.2,.5.4,0.)3.8%,"since November 2020, Russia has deployed about...",men age 18-35 years for compulsory military se...
20,Bangladesh,"24 00 N, 90 00 E",Asia,"148,460 sq km",24 00 N,90 00 E,Harakat ul-Jihad-i-Islami/Bangladesh; Islamic ...,90 00 E,793.49,775.08,...,2.09,3.6,3.6,34.7,41172346,1.2,.1.3,0.)1.4%,the military’s chief areas of focus are border...,16-21 years of age for voluntary military serv...
23,Belgium,"50 50 N, 4 00 E",Europe,"30,528 sq km",50 50 N,4 00 E,Islamic Revolutionary Guard Corps/Qods Force; ...,4 00 E,557.11,594.47,...,1.77,22.1,22.1,23.4,10620701,1.1,.1,0.9%,Belgium is a member of NATO and was one of the...,18 years of age for male and female voluntary ...


In [196]:
#export to CSV

df_d.to_csv("first_pass_clean_cia.csv")